In [15]:
#generate dataset
#train the classifier and save it
#detect the face and named it if is already stored data set

generate dataset

In [17]:
import cv2
import os

def generate_dataset():
    # Load Haar cascade properly
    face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

    # Verify cascade loaded correctly
    if face_classifier.empty():
        print("Error: Haar cascade file not found! Check OpenCV installation.")
        return

    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)  # Detect faces

        if len(faces) == 0:
            return None  # No face detected
        
        for (x, y, w, h) in faces:
            return img[y:y+h, x:x+w]  # Crop and return face

    cap = cv2.VideoCapture(0)
    id1 = 1
    img_id = 0

    if not os.path.exists("data"):
        os.makedirs("data")

    while True:
        ret, frame = cap.read()
        if not ret:
            continue

        face = face_cropped(frame)
        
        if face is not None:
            img_id += 1
            face = cv2.resize(face, (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            file_name_path = f"data/user.{id1}.{img_id}.jpg"
            cv2.imwrite(file_name_path, face)

            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow("Cropped Face", face)

        if cv2.waitKey(1) == 13 or img_id == 200:
            break

    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples completed...")

#generate_dataset()

#train the classifier and save it

In [1]:
import os
import cv2
import numpy as np
from PIL import Image

def train_classifier(data_dir): 
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')  # Convert to grayscale
        imageNp = np.array(img, 'uint8')  # Correct dtype
        id = int(os.path.split(image)[1].split(".")[1])  # Extract ID from filename
        faces.append(imageNp)
        ids.append(id)

    ids = np.array(ids)

    # Train the classifier and save
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")

# Call the function
train_classifier("data")

##detect the face and named it if is already stored data set

In [4]:
import cv2
import numpy as np
from PIL import Image
import os



In [1]:
import cv2

def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_image, scaleFactor, minNeighbors)

    coords = [] 

    for (x, y, w, h) in features:
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        id, pred = clf.predict(gray_image[y:y+h, x:x+w])
        confidence = int(100 * (1 - pred / 300))

        if confidence > 77: 
            if id == 1:
                cv2.putText(img, "Bhumika", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            elif id == 2:
                cv2.putText(img, "Neha", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        else: 
            cv2.putText(img, "UNKNOWN", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)

        coords = [x, y, w, h]  

    return coords  # Ensure it returns coords after the loop

def recognize(img, clf, faceCascade): 
    coords = draw_boundary(img, faceCascade, 1.1, 10, (255, 255, 255), "face", clf)
    return img

# Load Haarcascade Classifier
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")  # Corrected filename
# Load the trained recognizer model
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")

# Start video capture
video_capture = cv2.VideoCapture(0)  # Corrected capitalization

while True: 
    ret, img = video_capture.read()  # Corrected missing newline
    if not ret:
        break
    
    img = recognize(img, clf, faceCascade)
    cv2.imshow("Face Detection", img)

    if cv2.waitKey(1) == 13:  # Press 'Enter' (ASCII 13) to exit
        break

video_capture.release()
cv2.destroyAllWindows()

